In [1]:
import pandas as pd

### Probing Data

In [9]:
## probingして確実に1/0とわかっているもの
# https://docs.google.com/spreadsheets/d/1HOT8RaUxxFriDvzG_Z8E2nRLT1upZTfZHbNnaHpzmAI/edit#gid=0
df_probing = pd.read_csv('probing.csv')\
                        .loc[:, ['TransactionID', 'data_type', 'Probing_isFraud']]
df_probing.head()

,TransactionID,data_type,Probing_isFraud
0,3213891,train,NaN
1,3668748,test_public,1.0
2,3837824,test_private,1.0
3,3249556,train,NaN
4,3330018,train,NaN


In [10]:
## test_publicがないため、trainとのログを見比べて1だと判断したもの(精度やや低い)
# https://docs.google.com/spreadsheets/d/1HOT8RaUxxFriDvzG_Z8E2nRLT1upZTfZHbNnaHpzmAI/edit#gid=1004413265
# (CSVの元ファイルは"シート4")
df_probing2 = pd.read_csv('no_probing.csv')\
                          .loc[:, ['TransactionID', 'data_type', 'Probing_isFraud']]  # probingしてないのにカラム名がprobing_XXなのはご愛嬌
df_probing2.head()

,TransactionID,data_type,Probing_isFraud
0,3849686,test_private,1
1,4122243,test_private,1
2,4136765,test_private,1
3,4137063,test_private,1
4,4062040,test_private,1


In [11]:
#df_probing = pd.concat([df_probing, df_probing2])

### Kernel
* https://www.kaggle.com/kyakovlev/ieee-lgbm-with-groupkfold-cv
    * LB: 0.9485

In [12]:
df_konstantin = pd.read_csv('konstantin.csv').rename(columns={'isFraud': 'kernel_pred'})
df_konstantin.head()

,TransactionID,kernel_pred
0,3663549,0.000513
1,3663550,0.001965
2,3663551,0.001127
3,3663552,0.000770
4,3663553,0.001178


### Merge And Evaluation

In [13]:
df = pd.merge(df_probing, df_konstantin, on='TransactionID', how='left')
df.head()

,TransactionID,data_type,Probing_isFraud,kernel_pred
0,3213891,train,NaN,NaN
1,3668748,test_public,1.0,0.969774
2,3837824,test_private,1.0,0.991106
3,3249556,train,NaN,NaN
4,3330018,train,NaN,NaN


#### Test Public (LB)

In [14]:
from sklearn.metrics import roc_auc_score

roc_auc_score(
    df[df.data_type=="test_public"]['Probing_isFraud'],
    df[df.data_type=="test_public"]['kernel_pred']
)

0.960578971654658

#### Test Private

In [15]:
# df_probing2 を使わないときは 0.955
roc_auc_score(
    df[df.data_type=="test_private"]['Probing_isFraud'],
    df[df.data_type=="test_private"]['kernel_pred']
)

0.9554001956431523